# side model feature selection

In [1]:
import pandas as pd

df_features = pd.read_parquet("data/features.parquet")
print(df_features.shape)
df_features.isna().sum(axis=0).sort_values(ascending=False)

(13405, 3447)


adx_7                               0
mod_rsi_lag17                       0
mod_rsi_lag19                       0
mod_rsi_dt_lag1                     0
mod_rsi_dt_lag2                     0
                                   ..
comb_spectrum_dom_cycle_ddt_lag5    0
comb_spectrum_dom_cycle_ddt_lag6    0
comb_spectrum_dom_cycle_ddt_lag7    0
comb_spectrum_dom_cycle_ddt_lag8    0
williams_r_ddt_lag19                0
Length: 3447, dtype: int64

In [2]:
import numpy as np

label = np.load("data/side_label.npy")

print(label.shape)

len_gap = len(label) - len(df_features)

label = label[len_gap:]

print(label.shape)

(13469,)
(13405,)


In [3]:
from custom_indicators.toolbox.feature_selection.rfcq_selector import RFCQSelector

selector = RFCQSelector()
selector.fit(df_features, label)

➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性(使用随机森林)...
✓ 选择第1个特征: williams_r (最大重要性: 0.0457)
➤ 计算特征冗余度...
➤ 总计选择689个特征 (已选择1个，还需选择688个)...
➤ 开始MRMR迭代选择过程...


选择特征:   0%|                                                           | 0/688 [00:00<?, ?特征/s]


✅ 特征选择完成：从3447个特征中选择了689个，舍弃了2758个
✅ 选择的特征: ['williams_r', 'adaptive_cci_ddt', 'adx_14_dt', 'ehlers_early_onset_trend_ddt', 'vwap_dt_lag1', 'cmma', 'ehlers_early_onset_trend_ddt_lag1', 'vwap_dt', 'evenbetter_sinewave_short_dt', 'fisher_dt', 'cmma_lag1', 'roofing_filter_ddt', 'trendflex_dt_lag1', 'dual_diff_dt', 'stc', 'stc_dt', 'roofing_filter_ddt_lag1', 'norm_on_balance_volume', 'cmma_dt', 'evenbetter_sinewave_long_dt', 'iqr_ratio_dt', 'trendflex_dt', 'evenbetter_sinewave_short', 'voss_filt_dt', 'roofing_filter_dt', 'fisher_ddt', 'evenbetter_sinewave_short_dt_lag1', 'fisher_ddt_lag1', 'adx_7_dt', 'reactivity', 'fisher_dt_lag1', 'adaptive_cci_dt', 'acc_swing_index_dt', 'conv_0', 'trendflex_ddt_lag1', 'adaptive_stochastic_dt', 'evenbetter_sinewave_long', 'adaptive_bp', 'williams_r_dt', 'voss', 'stc_dt_lag1', 'williams_r_lag1', 'adaptive_stochastic_ddt', 'reactivity_dt', 'bandpass_dt', 'voss_dt_lag1', 'fisher', 'cmma_dt_lag2', 'adx_7_dt_lag1', 'aroon_diff', 'acc_swing_index_dt_lag1', 'a

In [4]:
side_res_long = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)
print((side_res_long > 0).sum())
side_res_long

487


williams_r              0.045697
cmma                    0.037757
fisher_dt               0.033824
vwap_dt_lag1            0.028390
trendflex_dt            0.027166
                          ...   
conv_22                 0.000000
conv_23                 0.000000
conv_24                 0.000000
conv_25                 0.000000
williams_r_ddt_lag19    0.000000
Length: 3447, dtype: float64

In [5]:
import json

feature_info = {
    "side": {
        "side": [],
    },
    "meta": {
        "meta": [],
        "model": [],
    },
}

feature_long = side_res_long[side_res_long > 0].index.tolist()


feature_info["side"]["side"] = feature_long


with open("strategies/BinanceBtcEntropyBarV1/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=4)

# meta model feature selection

In [6]:
import numpy as np
import pandas as pd

df_features = pd.read_parquet("data/features.parquet")
meta_label = np.load("data/label_meta.npy")
print(df_features.shape)
print(meta_label.shape)

df_features.isna().sum(axis=0).sort_values(ascending=False)

(13405, 3447)
(13405,)


adx_7                               0
mod_rsi_lag17                       0
mod_rsi_lag19                       0
mod_rsi_dt_lag1                     0
mod_rsi_dt_lag2                     0
                                   ..
comb_spectrum_dom_cycle_ddt_lag5    0
comb_spectrum_dom_cycle_ddt_lag6    0
comb_spectrum_dom_cycle_ddt_lag7    0
comb_spectrum_dom_cycle_ddt_lag8    0
williams_r_ddt_lag19                0
Length: 3447, dtype: int64

In [7]:
from strategies.BinanceBtcEntropyBarV1.config import (
    SIDE,
    get_side_model,
)

side_model = get_side_model(False)

side_res = side_model.predict(df_features[SIDE])

assert df_features.shape[0] == len(side_res)

df_features["model"] = side_res

In [8]:
# meta feature selection
from custom_indicators.toolbox.feature_selection.rfcq_selector import RFCQSelector

selector = RFCQSelector()
selector.fit(df_features, meta_label)
meta_res = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)

➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性(使用随机森林)...
✓ 选择第1个特征: corwin_schultz_estimator (最大重要性: 0.0551)
➤ 计算特征冗余度...
➤ 总计选择689个特征 (已选择1个，还需选择688个)...
➤ 开始MRMR迭代选择过程...


选择特征:   0%|                                                           | 0/688 [00:00<?, ?特征/s]


✅ 特征选择完成：从3448个特征中选择了689个，舍弃了2759个
✅ 选择的特征: ['corwin_schultz_estimator', 'model', 'bekker_parkinson_vol_ddt', 'adx_14_ddt', 'norm_on_balance_volume_ddt', 'conv_0', 'amihud_lambda_dt', 'williams_r', 'price_variance_ratio_ddt', 'phase_accumulation_dt_lag19', 'bekker_parkinson_vol_dt', 'trendflex_ddt', 'adx_7_dt', 'cmma_ddt', 'pfe_dt_lag19', 'dual_diff_ddt', 'cmma', 'reactivity_lag17', 'adx_7_ddt', 'bekker_parkinson_vol', 'acc_swing_index_dt', 'price_change_oscillator_dt', 'corwin_schultz_estimator_dt', 'hurst_coef_slow_dt_lag18', 'hasbrouck_lambda_dt', 'fti_dt', 'bandpass_ddt', 'aroon_diff_dt', 'price_variance_ratio_dt', 'forecast_oscillator', 'vwap_dt', 'acp_pwr_11', 'vwap_dt_lag2', 'roll_impact_dt', 'acc_swing_index_ddt', 'kyle_lambda', 'adaptive_cci_ddt_lag5', 'dft_dom_cycle_dt_lag8', 'natr_dt', 'evenbetter_sinewave_short_ddt', 'adx_14_dt', 'dft_dom_cycle_ddt_lag18', 'vwap_dt_lag1', 'evenbetter_sinewave_long_ddt_lag16', 'comb_spectrum_dom_cycle_dt_lag2', 'cmma_lag16', 'norm_on_balanc

In [9]:
(meta_res > 0).sum()

383

In [10]:
import json
from pathlib import Path

feature_info = Path("strategies/BinanceBtcEntropyBarV1/feature_info.json")
if feature_info.exists():
    with open(feature_info, "r") as f:
        feature_info = json.load(f)

feature_info["meta"]["meta"] = []
feature_info["meta"]["model"] = []

meta_features = meta_res[meta_res > 0].index.tolist()
for k in meta_features:
    if k == "model":
        feature_info["meta"]["model"].append(k)
    else:
        feature_info["meta"]["meta"].append(k)


with open("strategies/BinanceBtcEntropyBarV1/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=4)